In [1]:
import pm4py.algo.discovery.log_skeleton.algorithm as lsk_check
import pm4py.algo.conformance.log_skeleton.algorithm
from templates import TEMPLATE
from pycelonis.celonis_api.pql.pql import PQL, PQLColumn, PQLFilter


def variant_table(datamodel, table: str):
    """

    @param datamode: the datamodel in celonis
    @param table: the selected table in celonis
    @return: the variant table(dataframe) for different traces, and the activities list
    """
    ## table for variant table
    pql = PQL()
    pql.add(
        PQLColumn(name="variant", query='VARIANT ( "' + table + '"."concept:name"  )')
    )
    pql.add(PQLColumn(name="freq", query='COUNT("' + table + '_CASES"."Case ID")'))

    pql.add(PQLColumn(name="id", query='MIN("' + table + '_CASES"."Case ID")'))
    df = datamodel.get_data_frame(pql)
    df.set_index('id', inplace=True)

    return df


def conformance_checking(datamodel, table: str, model):
    scc_df = stric_conformance(datamodel, table, model)
    v_df = variant_table(datamodel, table)

    return conf_df_to_dics(scc_df, v_df)


def get_act_order(acties, con, l):
    #     al = [a.strip() for a in  acties.split(',')]

    if ("not_in_model" in con):
        s = []
        for i in range(len(acties)):
            if acties[i] not in l:
                s.append(i)
        return s
    elif ("never_together" in con):
        b = con.split(' ')[-1]
        a = con.split(' ')[-3]
        if (a in acties and b in acties):
            return [acties.index(a), acties.index(b)]
        elif (a in acties):
            return [acties.index(a)]
        else:
            return [acties.index(b)]
    elif ('TO' not in con):
        a = con.split('_')[0]
        return [i for i, x in enumerate(acties) if x == a]
    else:
        b = con.split(' ')[-1]
        a = con.split(' ')[-3]
        return [acties.index(a), acties.index(b)]


def conf_df_to_dics(con_df, v_df):
    scc_df = con_df.set_index('id')
    cols = scc_df.iloc[:, 1:].columns

    l = set([l.strip() for l in ','.join(v_df['variant']).split(',')])

    sum_trace = int(sum(v_df['freq']))
    fulfill_trace = 0
    cc_dic = {}
    for id in scc_df.index.values.tolist():
        violations = []
        fulfill = []
        dic = {}
        acties = [s.strip() for s in scc_df.at[id, 'variant'].split(',')]
        dic['variant'] = acties
        for con in cols:
            if (scc_df.at[id, con] < 0):
                violations.append({con: sorted(get_act_order(acties, con, l))})
            if (scc_df.at[id, con] > 0):
                fulfill.append({con: (sorted(get_act_order(acties, con, l)), acties)})
        if (len(violations) == 0):
            fulfill_trace += v_df.at[id, 'freq']
        fulfill_trace = int(fulfill_trace)
        dic['violations'] = violations
        dic['fulfill'] = fulfill
        activations = int(sum(scc_df.loc[id][1:].map(abs)))
        fulfills = sum(scc_df.loc[id][1:])
        dev = int(activations - fulfills)

        dic['fitness'] = 1 - dev / activations
        print(type(dic['fitness']))
        dic['activations'] = activations
        dic['fit'] = (fulfill == activations)

        cc_dic[id] = dic
    vio_trace = sum_trace - fulfill_trace
    return cc_dic, [sum_trace, fulfill_trace, vio_trace]


def stric_conformance(datamodel, table, model):
    ## get the variant table
    v_df = variant_table(datamodel, table)
    variant_dic = v_df.groupby("id")["freq"].apply(int).to_dict()

    select_id = str("','".join(v_df.index.values.tolist()))
    filter_query = (
            'FILTER DOMAIN "synthetic_event_log_xes_CASES"."Case ID" IN ( \''
            + select_id
            + "' )"
    )
    pql = PQL()
    pql.add(PQLFilter(filter_query))
    pql.add(PQLColumn(name="id", query='"' + table + '_CASES"."Case Id"'))
    pql.add(PQLColumn(name='variant', query='VARIANT ( "' + table + '"."concept:name" ) '))
    # for key in model.keys():

    # always_after_set = model["always_after"]
    for activities in model[TEMPLATE.Always_After.value]:
        pql += conformance_always_after(activities)
    for activities in model[TEMPLATE.Always_Before.value]:
        pql += conformance_always_before(activities)
    for activities in model[TEMPLATE.Never_Together.value]:
        pql += conformance_never_together(activities, table)
    for activities in model[TEMPLATE.Directly_Follows.value]:
        pql += conformance_directly_follow(activities)
    for activities in model[TEMPLATE.Equivalence.value]:
        pql += conformance_equivalence(activities, table)
    for pqlc in conformance_frequency(model['activ_freq'], table):
        pql += pqlc
    df = datamodel.get_data_frame(pql)
    return df
    # for pair in always_after_set:


def conformance_always_after(activities):
    activities = list(activities)
    a = str(activities[0])
    b = str(activities[1])

    return PQLColumn(
        name="always_after " + a + " TO " + b,
        query="CASE WHEN PROCESS NOT EQUALS '"
              + a
              + "' THEN 0 WHEN PROCESS EQUALS '"
              + a
              + "' TO ANY TO '"
              + b
              + "' THEN 1 ELSE -1 END",
    )


def conformance_always_before(activities):
    activities = list(activities)
    a = str(activities[0])
    b = str(activities[1])

    return PQLColumn(
        name="always_before " + a + " TO " + b,
        query="CASE WHEN PROCESS NOT EQUALS '"
              + a
              + "' THEN 0 WHEN PROCESS EQUALS '"
              + b
              + "' TO ANY TO '"
              + a
              + "' THEN 1 ELSE -1 END",
    )


def conformance_never_together(activities, table):
    activities = list(activities)
    a = str(activities[0])
    b = str(activities[1])
    query = (
            "CASE WHEN PROCESS NOT EQUALS '"
            + a
            + "'THEN 0 WHEN MATCH_ACTIVITIES(\""
            + table
            + '"."concept:name", NODE[\''
            + a
            + "'],EXCLUDING['"
            + b
            + "']) = 1 THEN 1 ELSE -1 END"
    )
    return PQLColumn(name="never_together " + a + " TO " + b, query=query)


def conformance_directly_follow(activities):
    activities = list(activities)
    a = str(activities[0])
    b = str(activities[1])
    query = (
            "CASE WHEN PROCESS NOT EQUALS '"
            + a
            + "' THEN 0 WHEN PROCESS EQUALS '"
            + a
            + "' TO '"
            + b
            + "' THEN 1 ELSE -1 END"
    )
    return PQLColumn(name="directly_follow " + a + " TO " + b, query=query)


def conformance_frequency(activity, table):
    all_act_in_model = "('" + "','".join(list(activity.keys())) + "')"
    print(all_act_in_model)
    pql_list = []
    pql_list.append(PQLColumn(name='not_in_model',
                              query='SUM(CASE WHEN ("' + table + '"."concept:name") in ' + all_act_in_model + "THEN 0 ELSE -1 END)"))
    for ac in activity.keys():
        calc_reworl = 'CALC_REWORK("' + table + '"."concept:name" IN (\'' + ac + '\'))'
        freq = "(" + ",".join(map(str, list(activity[ac]))) + ")"
        query = "CASE WHEN " + calc_reworl + "=0 AND 0 IN " + freq + " THEN 0 WHEN " + calc_reworl + ">0 AND " + calc_reworl + " IN " + freq + "THEN 1 ELSE -1 END"
        #         print(calc_reworl)
        #         print(freq)
        #         print(query)
        pql_list.append(PQLColumn(name=ac + "_" + freq, query=query))
    return pql_list


def conformance_equivalence(activities, table):
    activities = list(activities)
    a = str(activities[0])
    b = str(activities[1])

    def sum_query(a):
        return (
                'SUM( CASE WHEN "'
                + table
                + '"."concept:name" = \''
                + a
                + "' THEN 1 ELSE 0 END)"
        )

    query = (
            "CASE WHEN "
            + sum_query(a)
            + "  = 0 "
            + "THEN 0 WHEN "
            + sum_query(a)
            + " = "
            + sum_query(b)

            + "THEN 1 ELSE -1 END"
    )
    col_name = "equivalence " + a + " TO " + b
    return (PQLColumn(name=col_name, query=query))

In [2]:
from celonis_connect import Celonis_Connect
from discovery.model_discover import declare_model_discover

cn = Celonis_Connect()
datamodel = cn.get_datamodels().find("synthetic")
table = "synthetic_event_log_xes"
model = declare_model_discover(datamodel,table,0)

/Users/baichaoye/opt/anaconda3/envs/lab/lib/python3.9/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 1.7.1#sha256=1a33bea4a3fe5bf28b39faa561448594f768b243c3260800b7bddf5b80b46d2e is an invalid version and will not be supported in a future release
  warnings.warn(
/Users/baichaoye/opt/anaconda3/envs/lab/lib/python3.9/site-packages/pkg_resources/_vendor/packaging/version.py:111: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  warnings.warn(


[2022-07-04 07:12:09] INFO: Initial connect successful! Hello Baichao. PyCelonis Version: 1.7.1
[2022-07-04 07:12:12] INFO: PQL Export started...
[2022-07-04 07:12:12] INFO: PQL Export status: DONE
[2022-07-04 07:12:12] INFO: PQL Export started...
[2022-07-04 07:12:13] INFO: PQL Export status: DONE
[2022-07-04 07:12:14] INFO: PQL Export started...
[2022-07-04 07:12:15] INFO: PQL Export status: DONE
[2022-07-04 07:12:16] INFO: PQL Export started...
[2022-07-04 07:12:16] INFO: PQL Export status: DONE
[2022-07-04 07:12:16] INFO: PQL Export started...
[2022-07-04 07:12:16] INFO: PQL Export status: DONE
[2022-07-04 07:12:17] INFO: PQL Export started...
[2022-07-04 07:12:18] INFO: PQL Export status: DONE
[2022-07-04 07:12:19] INFO: PQL Export started...
[2022-07-04 07:12:19] INFO: PQL Export status: DONE


In [4]:
for key in model['activ_freq']:
    print(type(model['activ_freq'][key]))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [5]:
z = json.dumps(model)
h = open('z.json','w')
h.write(z)
h.close()

In [6]:
with open ('z.json','r') as j:
    j_z = json.loads(j.read())
j_z

{'equivalence': [['a', 'c'],
  ['b', 'c'],
  ['d', 'c'],
  ['e', 'a'],
  ['e', 'b'],
  ['e', 'c'],
  ['e', 'f'],
  ['f', 'c'],
  ['x', 'a'],
  ['x', 'b'],
  ['x', 'c'],
  ['x', 'e'],
  ['x', 'f']],
 'always_after': [['a', 'c'], ['e', 'f'], ['x', 'e'], ['x', 'f']],
 'always_before': [['d', 'c'],
  ['e', 'a'],
  ['e', 'b'],
  ['e', 'c'],
  ['f', 'c'],
  ['x', 'a'],
  ['x', 'b'],
  ['x', 'c']],
 'never_together': [['d', 'x'], ['x', 'd']],
 'directly_follows': [['e', 'f'], ['x', 'e']],
 'activ_freq': {'a': [0, 1],
  'b': [0, 1],
  'c': [1],
  'd': [0, 1],
  'e': [0, 1],
  'f': [0, 1],
  'x': [0, 1]}}

In [15]:
 model1['equivalence'] in model['equivalence']

False

In [7]:
with open("./static/failed.json",'r') as j:
    model1 = json.loads(j.read())
model1

{'equivalence': [['a', 'c'],
  ['b', 'c'],
  ['d', 'c'],
  ['e', 'a'],
  ['e', 'b'],
  ['e', 'c'],
  ['e', 'f'],
  ['f', 'c'],
  ['x', 'a'],
  ['x', 'b'],
  ['x', 'c'],
  ['x', 'e'],
  ['x', 'f']],
 'always_after': [['a', 'c'], ['e', 'f'], ['x', 'e'], ['x', 'f']],
 'always_before': [['d', 'c'],
  ['e', 'a'],
  ['e', 'b'],
  ['e', 'c'],
  ['f', 'c'],
  ['x', 'a'],
  ['x', 'b'],
  ['x', 'c']],
 'never_together': [['d', 'x'], ['x', 'd']],
 'directly_follows': [['e', 'f'], ['x', 'e']],
 'activ_freq': {'a': '[0, 1]',
  'b': '[0, 1]',
  'c': '[1]',
  'd': '[0, 1]',
  'e': '[0, 1]',
  'f': '[0, 1]',
  'x': '[0, 1]'}}

In [8]:
con_dic,statis = conformance_checking(datamodel,table,j_z)
con_dic

[2022-07-04 07:16:01] INFO: PQL Export started...
[2022-07-04 07:16:02] INFO: PQL Export status: DONE
('a','b','c','d','e','f','x')
[2022-07-04 07:16:03] INFO: PQL Export started...
[2022-07-04 07:16:03] INFO: PQL Export status: DONE
[2022-07-04 07:16:04] INFO: PQL Export started...
[2022-07-04 07:16:05] INFO: PQL Export status: DONE
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>


{'0040cb14-bbf7-4fc7-a345-b8c79aebf87c': {'variant': ['a', 'b', 'c', 'd', 'f'],
  'violations': [],
  'fulfill': [{'always_after a TO c': ([0, 2], ['a', 'b', 'c', 'd', 'f'])},
   {'always_before d TO c': ([2, 3], ['a', 'b', 'c', 'd', 'f'])},
   {'always_before f TO c': ([2, 4], ['a', 'b', 'c', 'd', 'f'])},
   {'never_together d TO x': ([3], ['a', 'b', 'c', 'd', 'f'])},
   {'equivalence a TO c': ([0, 2], ['a', 'b', 'c', 'd', 'f'])},
   {'equivalence b TO c': ([1, 2], ['a', 'b', 'c', 'd', 'f'])},
   {'equivalence d TO c': ([2, 3], ['a', 'b', 'c', 'd', 'f'])},
   {'equivalence f TO c': ([2, 4], ['a', 'b', 'c', 'd', 'f'])},
   {'a_(0,1)': ([0], ['a', 'b', 'c', 'd', 'f'])},
   {'b_(0,1)': ([1], ['a', 'b', 'c', 'd', 'f'])},
   {'c_(1)': ([2], ['a', 'b', 'c', 'd', 'f'])},
   {'d_(0,1)': ([3], ['a', 'b', 'c', 'd', 'f'])},
   {'f_(0,1)': ([4], ['a', 'b', 'c', 'd', 'f'])}],
  'fitness': 1.0,
  'activations': 13,
  'fit': False},
 '006825a4-f9e8-4a92-8343-b0e593210c7b': {'variant': ['a',
   'b',


In [4]:
statis

[1100, 1100, 0]

In [5]:
con_df = stric_conformance(datamodel,table,model)

[2022-07-03 17:45:53] INFO: PQL Export started...
[2022-07-03 17:45:53] INFO: PQL Export status: DONE
('a','b','c','d','e','f','x')
[2022-07-03 17:45:54] INFO: PQL Export started...
[2022-07-03 17:45:54] INFO: PQL Export status: DONE


In [6]:
conf_df_to_dics(con_df,v_df = variant_table(datamodel,table))

[2022-07-03 17:45:54] INFO: PQL Export started...
[2022-07-03 17:45:54] INFO: PQL Export status: DONE
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>


({'0040cb14-bbf7-4fc7-a345-b8c79aebf87c': {'variant': ['a',
    'b',
    'c',
    'd',
    'f'],
   'violations': [],
   'fulfill': [{'always_after a TO c': ([0, 2], ['a', 'b', 'c', 'd', 'f'])},
    {'always_before d TO c': ([2, 3], ['a', 'b', 'c', 'd', 'f'])},
    {'always_before f TO c': ([2, 4], ['a', 'b', 'c', 'd', 'f'])},
    {'never_together d TO x': ([3], ['a', 'b', 'c', 'd', 'f'])},
    {'equivalence a TO c': ([0, 2], ['a', 'b', 'c', 'd', 'f'])},
    {'equivalence b TO c': ([1, 2], ['a', 'b', 'c', 'd', 'f'])},
    {'equivalence d TO c': ([2, 3], ['a', 'b', 'c', 'd', 'f'])},
    {'equivalence f TO c': ([2, 4], ['a', 'b', 'c', 'd', 'f'])},
    {'a_(0,1)': ([0], ['a', 'b', 'c', 'd', 'f'])},
    {'b_(0,1)': ([1], ['a', 'b', 'c', 'd', 'f'])},
    {'c_(1)': ([2], ['a', 'b', 'c', 'd', 'f'])},
    {'d_(0,1)': ([3], ['a', 'b', 'c', 'd', 'f'])},
    {'f_(0,1)': ([4], ['a', 'b', 'c', 'd', 'f'])}],
   'fitness': 1.0,
   'activations': 13,
   'fit': False},
  '006825a4-f9e8-4a92-8343-b0e593

In [7]:

def conf_df_to_dics(con_df, v_df):
    scc_df = con_df.set_index('id')
    cols = scc_df.iloc[:, 1:].columns

    l = set([l.strip() for l in ','.join(v_df['variant']).split(',')])

    sum_trace = int(sum(v_df['freq']))
    fulfill_trace = 0
    cc_dic = {}
    for id in scc_df.index.values.tolist():
        violations = []
        fulfill = []
        dic = {}
        acties = [s.strip() for s in scc_df.at[id, 'variant'].split(',')]
        dic['variant'] = acties
        for con in cols:
            if (scc_df.at[id, con] < 0):
                violations.append({con: sorted(get_act_order(acties, con, l))})
            if (scc_df.at[id, con] > 0):
                fulfill.append({con: (sorted(get_act_order(acties, con, l)), acties)})
        if (len(violations) == 0):
            fulfill_trace += v_df.at[id, 'freq']
        fulfill_trace = int(fulfill_trace)
        dic['violations'] = violations
        dic['fulfill'] = fulfill
        activations = int(sum(scc_df.loc[id][1:].map(abs)))
        fulfills = sum(scc_df.loc[id][1:])
        dev = int(activations - fulfills)
        
        dic['fitness'] = 1 - dev / activations
        print(type(dic['fitness']))
        dic['activations'] = activations
        dic['fit'] = (fulfill == activations)

        cc_dic[id] = dic
    vio_trace = sum_trace - fulfill_trace
    return cc_dic, [sum_trace, fulfill_trace, vio_trace]

In [9]:
import json
d = json.dumps(con_dic)
g = open("con.json",'w')
g.write(d)
g.close()

In [9]:
con_df
print(con_df.columns)

Index(['id', 'variant', 'always_after a TO c', 'always_after e TO f',
       'always_after x TO e', 'always_after x TO f', 'always_before d TO c',
       'always_before e TO a', 'always_before e TO b', 'always_before e TO c',
       'always_before f TO c', 'always_before x TO a', 'always_before x TO b',
       'always_before x TO c', 'never_together d TO x',
       'never_together x TO d', 'directly_follow e TO f',
       'directly_follow x TO e', 'equivalence a TO c', 'equivalence b TO c',
       'equivalence d TO c', 'equivalence e TO a', 'equivalence e TO b',
       'equivalence e TO c', 'equivalence e TO f', 'equivalence f TO c',
       'equivalence x TO a', 'equivalence x TO b', 'equivalence x TO c',
       'equivalence x TO e', 'equivalence x TO f', 'not_in_model', 'a_(0,1)',
       'b_(0,1)', 'c_(1)', 'd_(0,1)', 'e_(0,1)', 'f_(0,1)', 'x_(0,1)'],
      dtype='object')


In [10]:
v_df = variant_table(datamodel,table)
v_df

[2022-07-03 17:45:55] INFO: PQL Export started...
[2022-07-03 17:45:55] INFO: PQL Export status: DONE


,variant,freq
id,,
00d9a426-3496-43dc-be2d-069b04490b0d,"a, b, c, d",45
00b06e7d-2c16-4808-a003-e690cdd173db,"a, b, c, d, e, f",40
0040cb14-bbf7-4fc7-a345-b8c79aebf87c,"a, b, c, d, f",200
0131bf9c-e7f4-45ac-8435-a07eee9f69ba,"a, b, c, e, f",200
0653126e-3746-479f-821f-c7ffa28f968c,"a, b, c, f",30
006825a4-f9e8-4a92-8343-b0e593210c7b,"a, b, c, x, e, f",100
01b925fc-7b55-4e18-ad36-e8f1c0242608,"a, c, b, d, f",200
01e84bd3-a6e0-40ab-b478-35271c007cd2,"a, c, b, e, f",200
03027514-4850-4fcc-836b-d6ded96f7604,"a, c, d, f",50


In [11]:
sum(v_df['freq'])

1100

In [12]:
l = []
set([l.strip() for l in ','.join(v_df['variant']).split(',')])

{'a', 'b', 'c', 'd', 'e', 'f', 'x'}

In [13]:
get_act_order(['1','2','3','1'],"1_d")

TypeError: get_act_order() missing 1 required positional argument: 'l'

In [ ]:
get_act_order(['1','2','3','1'],"d 2 TO 1")

In [ ]:
l = set([l.strip() for l in ','.join(v_df['variant']).split(',')])
l

In [ ]:

acties = [s.strip() for s in "a,y,h,c".split(',')]
get_act_order(acties,'never_together y TO c',l)

In [ ]:
'not_in_model' in "not_in_model"

In [ ]:
def get_act_order(acties,con,l):
#     al = [a.strip() for a in  acties.split(',')]
    
    if ("not_in_model" in con):
        s = []
        for i in range(len(acties)):
            if acties[i] not in l:
                s.append(i)
        return s
    elif("never_together" in con):
        b = con.split(' ')[-1]
        a = con.split(' ')[-3]
        if(a in acties and b in acties):
            return [acties.index(a),acties.index(b)]
        elif(a in acties):
            return [acties.index(a)]
        else:
            return [acties.index(b)]
    elif ('TO' not in con):
        a = con.split('_')[0]
        return [i for i, x in enumerate(acties) if x == a]
    else:
        b = con.split(' ')[-1]
        a = con.split(' ')[-3]
        return[acties.index(a),acties.index(b)]
    

def conf_df_to_dics(con_df, v_df):
    scc_df = con_df.set_index('id')
    cols = scc_df.iloc[:, 1:].columns
    
    l = set([l.strip() for l in ','.join(v_df['variant']).split(',')])
    
    
    
    sum_trace = sum(v_df['freq'])
    fulfill_trace = 0
    cc_dic = {}
    for id in scc_df.index.values.tolist():
        violations = []
        fulfill = []
        dic = {}
        acties = [s.strip() for s in scc_df.at[id,'variant'].split(',')]
        dic['variant'] = acties
        for con in cols:
            if (scc_df.at[id, con] < 0):
                
                violations.append({con:sorted(get_act_order(acties,con,l))})
            if (scc_df.at[id, con] > 0):
                
                fulfill.append({con:(sorted(get_act_order(acties,con,l)),acties)})
        if(len(violations) == 0):
            fulfill_trace += v_df.at[id,'freq']
        
        dic['violations'] = violations
        dic['fulfill'] = fulfill
        activations = sum(scc_df.loc[id][1:].map(abs))
        fulfills = sum(scc_df.loc[id][1:])
        dev = activations - fulfills
        dic['fitness'] = 1 - dev / activations
        dic['activations'] = activations
        dic['fit'] = (fulfill == activations)
        
        cc_dic[id] = dic
    vio_trace = sum_trace - fulfill_trace
    return cc_dic, [sum_trace,fulfill_trace,vio_trace]

In [14]:
dic,statis = conf_df_to_dics(con_df,v_df)

dic['0040cb14-bbf7-4fc7-a345-b8c79aebf87c']

<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>


{'variant': ['a', 'b', 'c', 'd', 'f'],
 'violations': [],
 'fulfill': [{'always_after a TO c': ([0, 2], ['a', 'b', 'c', 'd', 'f'])},
  {'always_before d TO c': ([2, 3], ['a', 'b', 'c', 'd', 'f'])},
  {'always_before f TO c': ([2, 4], ['a', 'b', 'c', 'd', 'f'])},
  {'never_together d TO x': ([3], ['a', 'b', 'c', 'd', 'f'])},
  {'equivalence a TO c': ([0, 2], ['a', 'b', 'c', 'd', 'f'])},
  {'equivalence b TO c': ([1, 2], ['a', 'b', 'c', 'd', 'f'])},
  {'equivalence d TO c': ([2, 3], ['a', 'b', 'c', 'd', 'f'])},
  {'equivalence f TO c': ([2, 4], ['a', 'b', 'c', 'd', 'f'])},
  {'a_(0,1)': ([0], ['a', 'b', 'c', 'd', 'f'])},
  {'b_(0,1)': ([1], ['a', 'b', 'c', 'd', 'f'])},
  {'c_(1)': ([2], ['a', 'b', 'c', 'd', 'f'])},
  {'d_(0,1)': ([3], ['a', 'b', 'c', 'd', 'f'])},
  {'f_(0,1)': ([4], ['a', 'b', 'c', 'd', 'f'])}],
 'fitness': 1.0,
 'activations': 13,
 'fit': False}

0040cb14-bbf7-4fc7-a345-b8c79aebf87c